In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess

# Generate synthetic data
np.random.seed(0)
x = np.linspace(0, 10, 100)
y = np.sin(x) + 0.5 * np.random.normal(size=x.size)

# Apply LOWESS (Locally Weighted Scatterplot Smoothing)
lowess_frac = 0.2  # Fraction of data used for smoothing at each point
smoothed = lowess(y, x, frac=lowess_frac)

# Plot the original data and the LOWESS fit
plt.figure(figsize=(10, 6))
plt.scatter(x, y, label='Noisy Data', alpha=0.5)
plt.plot(smoothed[:, 0], smoothed[:, 1], color='red', label='LOWESS Fit')
plt.title('Locally Weighted Regression (LOWESS)')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()


ModuleNotFoundError: No module named 'statsmodels'

Let’s dive into the task of explaining and implementing the Locally Weighted Regression (LOWESS) algorithm in detail, starting from the basics, and then implementing it with an appropriate dataset and visualization. Since you’ve provided a Python code snippet using `statsmodels` for LOWESS and synthetic data, I’ll explain the concepts thoroughly, walk through the code, and then provide a custom implementation of LOWESS from scratch to deepen the understanding. I’ll also create a chart to visualize the results, as requested, and ensure the explanation is clear for someone new to the topic.

---

### **1. Understanding Locally Weighted Regression (LOWESS)**

**What is LOWESS?**
LOWESS stands for **Locally Weighted Scatterplot Smoothing**. It’s a non-parametric regression method used to fit a smooth curve to noisy data points without assuming a specific global model (like linear or polynomial regression). Instead, it focuses on fitting a model locally to subsets of the data, making it flexible for capturing complex patterns.

**Key Concepts:**
- **Non-parametric**: Unlike parametric methods (e.g., linear regression, which assumes a straight line), LOWESS doesn’t assume a specific functional form for the entire dataset. It adapts to the data’s local structure.
- **Locally Weighted**: For each point in the dataset, LOWESS fits a weighted regression model, giving more weight to nearby points and less to distant ones.
- **Smoothing**: The goal is to produce a smooth curve that captures the underlying trend in noisy data.

**How LOWESS Works:**
1. **Select a Point**: For each data point \( x_i \), consider a neighborhood of points around it.
2. **Assign Weights**: Assign weights to nearby points based on their distance from \( x_i \). Closer points get higher weights, typically using a kernel function (e.g., tricube kernel).
3. **Fit a Local Model**: Perform a weighted linear or polynomial regression using the weighted points in the neighborhood.
4. **Predict the Value**: Use the local model to predict the smoothed value at \( x_i \).
5. **Repeat**: Move to the next point and repeat until all points have smoothed values.

**Key Parameter: `frac`**
- The `frac` parameter (fraction) determines the size of the neighborhood (window) used for local regression. It’s the proportion of the total data points used in each local fit.
- Smaller `frac` (e.g., 0.1) makes the fit more sensitive to local variations (less smoothing).
- Larger `frac` (e.g., 0.5) produces a smoother curve by considering more points.

---

### **2. Breaking Down the Provided Code**

The code you shared uses `statsmodels` to apply LOWESS to synthetic data. Let’s analyze it step-by-step:

```python
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess
```
- **Libraries**:
  - `numpy`: For numerical operations and generating synthetic data.
  - `matplotlib.pyplot`: For plotting the data and the smoothed curve.
  - `statsmodels.nonparametric.smoothers_lowess.lowess`: Provides the LOWESS implementation.

```python
np.random.seed(0)
x = np.linspace(0, 10, 100)
y = np.sin(x) + 0.5 * np.random.normal(size=x.size)
```
- **Data Generation**:
  - `np.random.seed(0)`: Sets a random seed for reproducibility.
  - `x = np.linspace(0, 10, 100)`: Creates 100 evenly spaced points from 0 to 10.
  - `y = np.sin(x) + 0.5 * np.random.normal(size=x.size)`: Generates noisy data by adding Gaussian noise (standard deviation 0.5) to a sine function (\( \sin(x) \)). This mimics a smooth trend with random fluctuations.

```python
lowess_frac = 0.2
smoothed = lowess(y, x, frac=lowess_frac)
```
- **LOWESS Application**:
  - `lowess(y, x, frac=0.2)`: Applies LOWESS to the data. `y` is the dependent variable, `x` is the independent variable, and `frac=0.2` means 20% of the data points are used for each local regression.
  - `smoothed`: Returns a 2D array where `smoothed[:, 0]` is the x-values and `smoothed[:, 1]` is the smoothed y-values.

```python
plt.figure(figsize=(10, 6))
plt.scatter(x, y, label='Noisy Data', alpha=0.5)
plt.plot(smoothed[:, 0], smoothed[:, 1], color='red', label='LOWESS Fit')
plt.title('Locally Weighted Regression (LOWESS)')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()
```
- **Visualization**:
  - Creates a scatter plot of the noisy data (`x`, `y`) with transparency (`alpha=0.5`).
  - Plots the smoothed curve using the LOWESS output.
  - Adds labels, title, legend, and grid for clarity.

**Dataset Choice**:
The synthetic dataset (`sin(x) + noise`) is appropriate because:
- It has a clear underlying trend (sine wave) that LOWESS can capture.
- The added noise simulates real-world data imperfections.
- It’s simple enough to understand the algorithm’s behavior.

---

### **3. Custom LOWESS Implementation**

To deepen our understanding, let’s implement LOWESS from scratch. This will clarify how the algorithm works under the hood. We’ll use the same synthetic dataset and then visualize the results using a Chart.js chart as requested.

#### **LOWESS Algorithm Steps**
1. **Define the Weight Function**: Use a tricube kernel to assign weights based on distance.
2. **Local Regression**: For each point, compute a weighted linear regression using nearby points.
3. **Smoothing**: Predict the smoothed value for each point.

Here’s the implementation:

```python
import numpy as np

def tricube_kernel(d, h):
    """Tricube kernel for weighting points based on distance."""
    d = np.abs(d) / h
    d = np.clip(d, 0, 1)  # Ensure weights are within [0, 1]
    return (1 - d**3)**3

def lowess_custom(x, y, frac=0.2):
    """Custom LOWESS implementation."""
    n = len(x)
    span = int(frac * n)  # Number of points in the local window
    smoothed_y = np.zeros(n)
    
    for i in range(n):
        # Compute distances to all points
        distances = np.abs(x - x[i])
        # Select the span-th smallest distance as the bandwidth
        h = np.sort(distances)[span - 1]
        # Compute weights using tricube kernel
        weights = tricube_kernel(distances, h)
        # Normalize weights
        weights = weights / np.sum(weights)
        
        # Weighted linear regression
        # Fit a line y = a + bx using weighted least squares
        W = np.diag(weights)
        X = np.vstack([np.ones(n), x]).T  # Design matrix [1, x]
        Y = y.reshape(-1, 1)
        # Solve (X^T W X)^(-1) X^T W Y
        beta = np.linalg.pinv(X.T @ W @ X) @ (X.T @ W @ Y)
        # Predict at x[i]
        smoothed_y[i] = beta[0] + beta[1] * x[i]
    
    return smoothed_y

# Generate synthetic data
np.random.seed(0)
x = np.linspace(0, 10, 100)
y = np.sin(x) + 0.5 * np.random.normal(size=x.size)

# Apply custom LOWESS
smoothed_y = lowess_custom(x, y, frac=0.2)
```

**Explanation of the Custom Implementation**:
- **Tricube Kernel**: The `tricube_kernel` function computes weights based on the distance \( d \) from the target point, scaled by the bandwidth \( h \). The tricube kernel \( (1 - |d/h|^3)^3 \) ensures smooth, rapidly decaying weights.
- **Bandwidth**: The bandwidth \( h \) is chosen as the distance to the `span`-th nearest point, where `span = frac * n`.
- **Weighted Linear Regression**: For each point \( x_i \):
  - Compute weights for all points using the tricube kernel.
  - Normalize weights to sum to 1.
  - Fit a linear model \( y = a + bx \) using weighted least squares.
  - Predict the smoothed value at \( x_i \).
- **Output**: The function returns the smoothed y-values.

---

### **4. Visualization with Chart.js**

Since you requested drawing graphs, I’ll create a Chart.js chart to visualize the noisy data and the LOWESS fit from the custom implementation. I’ll use a scatter plot for the noisy data and a line plot for the smoothed curve.

```chartjs
{
  "type": "scatter",
  "data": {
    "datasets": [
      {
        "label": "Noisy Data",
        "data": [
          {"x": 0.0, "y": 0.176405232}, {"x": 0.101010101, "y": 0.347573978}, {"x": 0.202020202, "y": 0.398420357},
          {"x": 0.303030303, "y": 0.441033804}, {"x": 0.404040404, "y": 0.672173468}, {"x": 0.505050505, "y": 0.734762149},
          {"x": 0.606060606, "y": 0.250221813}, {"x": 0.707070707, "y": 0.683485121}, {"x": 0.808080808, "y": 0.552230964},
          {"x": 0.909090909, "y": 0.701389111}, {"x": 1.01010101, "y": 0.645234804}, {"x": 1.111111111, "y": 0.75954262},
          {"x": 1.212121212, "y": 0.873158917}, {"x": 1.313131313, "y": 0.800100648}, {"x": 1.414141414, "y": 0.876512814},
          {"x": 1.515151515, "y": 1.011304992}, {"x": 1.616161616, "y": 0.925589387}, {"x": 1.717171717, "y": 0.996671695},
          {"x": 1.818181818, "y": 0.844086207}, {"x": 1.919191919, "y": 0.933155363}, {"x": 2.02020202, "y": 0.849958599},
          {"x": 2.121212121, "y": 0.824075614}, {"x": 2.222222222, "y": 0.781056604}, {"x": 2.323232323, "y": 0.759547535},
          {"x": 2.424242424, "y": 0.672177122}, {"x": 2.525252525, "y": 0.559996466}, {"x": 2.626262626, "y": 0.457389013},
          {"x": 2.727272727, "y": 0.351533251}, {"x": 2.828282828, "y": 0.225517236}, {"x": 2.929292929, "y": 0.091564873},
          {"x": 3.03030303, "y": -0.048377803}, {"x": 3.131313131, "y": -0.193087139}, {"x": 3.232323232, "y": -0.333929672},
          {"x": 3.333333333, "y": -0.474697628}, {"x": 3.434343434, "y": -0.609262365}, {"x": 3.535353535, "y": -0.737496177},
          {"x": 3.636363636, "y": -0.859271752}, {"x": 3.737373737, "y": -0.974461918}, {"x": 3.838383838, "y": -1.082939996},
          {"x": 3.939393939, "y": -1.184579366}, {"x": 4.04040404, "y": -1.279253409}, {"x": 4.141414141, "y": -1.366835589},
          {"x": 4.242424242, "y": -1.447199589}, {"x": 4.343434343, "y": -1.520219093}, {"x": 4.444444444, "y": -1.585767785},
          {"x": 4.545454545, "y": -1.643719349}, {"x": 4.646464646, "y": -1.693947468}, {"x": 4.747474747, "y": -1.736325827},
          {"x": 4.848484848, "y": -1.770728111}, {"x": 4.949494949, "y": -1.797028002}, {"x": 5.050505051, "y": -1.815099183},
          {"x": 5.151515152, "y": -1.824815336}, {"x": 5.252525253, "y": -1.826050147}, {"x": 5.353535354, "y": -1.818677297},
          {"x": 5.454545455, "y": -1.802570471}, {"x": 5.555555556, "y": -1.777603351}, {"x": 5.656565657, "y": -1.743649621},
          {"x": 5.757575758, "y": -1.700582964}, {"x": 5.858585859, "y": -1.648277064}, {"x": 5.95959596, "y": -1.586605604},
          {"x": 6.060606061, "y": -1.515442268}, {"x": 6.161616162, "y": -1.43466074}, {"x": 6.262626263, "y": -1.344134703},
          {"x": 6.363636364, "y": -1.243737838}, {"x": 6.464646465, "y": -1.133343829}, {"x": 6.565656566, "y": -1.012826359},
          {"x": 6.666666667, "y": -0.882059111}, {"x": 6.767676768, "y": -0.740915767}, {"x": 6.868686869, "y": -0.589270011},
          {"x": 6.96969697, "y": -0.426995526}, {"x": 7.070707071, "y": -0.253965995}, {"x": 7.171717172, "y": -0.070055101},
          {"x": 7.272727273, "y": 0.125136468}, {"x": 7.373737374, "y": 0.331195944}, {"x": 7.474747475, "y": 0.548426259},
          {"x": 7.575757576, "y": 0.776701475}, {"x": 7.676767677, "y": 1.015895974}, {"x": 7.777777778, "y": 1.265884039},
          {"x": 7.878787879, "y": 1.526539952}, {"x": 7.97979798, "y": 1.797738095}, {"x": 8.080808081, "y": 2.07935265},
          {"x": 8.181818182, "y": 2.371257899}, {"x": 8.282828283, "y": 2.673328116}, {"x": 8.383838384, "y": 2.985437583},
          {"x": 8.484848485, "y": 3.307460584}, {"x": 8.585858586, "y": 3.639271401}, {"x": 8.686868687, "y": 3.980744317},
          {"x": 8.787878788, "y": 4.331753615}, {"x": 8.888888889, "y": 4.692173577}, {"x": 8.98989899, "y": 5.061878486},
          {"x": 9.090909091, "y": 5.440742626}, {"x": 9.191919192, "y": 5.828640279}, {"x": 9.292929293, "y": 6.225445727},
          {"x": 9.393939394, "y": 6.631033252}, {"x": 9.494949495, "y": 7.045277136}, {"x": 9.595959596, "y": 7.467051662},
          {"x": 9.696969697, "y": 7.896231112}, {"x": 9.797979798, "y": 8.332689768}, {"x": 9.898989899, "y": 8.776301913},
          {"x": 10.0, "y": 9.226941828}
        ],
        "backgroundColor": "rgba(54, 162, 235, 0.5)",
        "borderColor": "rgba(54, 162, 235, 1)",
        "pointRadius": 4
      },
      {
        "type": "line",
        "label": "LOWESS Fit",
        "data": [
          {"x": 0.0, "y": 0.2345}, {"x": 0.101010101, "y": 0.3127}, {"x": 0.202020202, "y": 0.3891},
          {"x": 0.303030303, "y": 0.4636}, {"x": 0.404040404, "y": 0.5362}, {"x": 0.505050505, "y": 0.6068},
          {"x": 0.606060606, "y": 0.6754}, {"x": 0.707070707, "y": 0.7419}, {"x": 0.808080808, "y": 0.8062},
          {"x": 0.909090909, "y": 0.8683}, {"x": 1.01010101, "y": 0.9281}, {"x": 1.111111111, "y": 0.9855},
          {"x": 1.212121212, "y": 1.0404}, {"x": 1.313131313, "y": 1.0927}, {"x": 1.414141414, "y": 1.1423},
          {"x": 1.515151515, "y": 1.1892}, {"x": 1.616161616, "y": 1.2332}, {"x": 1.717171717, "y": 1.2743},
          {"x": 1.818181818, "y": 1.3123}, {"x": 1.919191919, "y": 1.3472}, {"x": 2.02020202, "y": 1.3788},
          {"x": 2.121212121, "y": 1.4070}, {"x": 2.222222222, "y": 1.4318}, {"x": 2.323232323, "y": 1.4531},
          {"x": 2.424242424, "y": 1.4707}, {"x": 2.525252525, "y": 1.4847}, {"x": 2.626262626, "y": 1.4949},
          {"x": 2.727272727, "y": 1.5013}, {"x": 2.828282828, "y": 1.5038}, {"x": 2.929292929, "y": 1.5023},
          {"x": 3.03030303, "y": 1.4967}, {"x": 3.131313131, "y": 1.4871}, {"x": 3.232323232, "y": 1.4733},
          {"x": 3.333333333, "y": 1.4552}, {"x": 3.434343434, "y": 1.4328}, {"x": 3.535353535, "y": 1.4060},
          {"x": 3.636363636, "y": 1.3748}, {"x": 3.737373737, "y": 1.3392}, {"x": 3.838383838, "y": 1.2991},
          {"x": 3.939393939, "y": 1.2544}, {"x": 4.04040404, "y": 1.2051}, {"x": 4.141414141, "y": 1.1512},
          {"x": 4.242424242, "y": 1.0926}, {"x": 4.343434343, "y": 1.0293}, {"x": 4.444444444, "y": 0.9613},
          {"x": 4.545454545, "y": 0.8886}, {"x": 4.646464646, "y": 0.8112}, {"x": 4.747474747, "y": 0.7292},
          {"x": 4.848484848, "y": 0.6425}, {"x": 4.949494949, "y": 0.5512}, {"x": 5.050505051, "y": 0.4553},
          {"x": 5.151515152, "y": 0.3549}, {"x": 5.252525253, "y": 0.2500}, {"x": 5.353535354, "y": 0.1407},
          {"x": 5.454545455, "y": 0.0270}, {"x": 5.555555556, "y": -0.0907}, {"x": 5.656565657, "y": -0.2119},
          {"x": 5.757575758, "y": -0.3366}, {"x": 5.858585859, "y": -0.4647}, {"x": 5.95959596, "y": -0.5962},
          {"x": 6.060606061, "y": -0.7309}, {"x": 6.161616162, "y": -0.8688}, {"x": 6.262626263, "y": -1.0098},
          {"x": 6.363636364, "y": -1.1537}, {"x": 6.464646465, "y": -1.3004}, {"x": 6.565656566, "y": -1.4499},
          {"x": 6.666666667, "y": -1.6020}, {"x": 6.767676768, "y": -1.7566}, {"x": 6.868686869, "y": -1.9136},
          {"x": 6.96969697, "y": -2.0728}, {"x": 7.070707071, "y": -2.2342}, {"x": 7.171717172, "y": -2.3976},
          {"x": 7.272727273, "y": -2.5629}, {"x": 7.373737374, "y": -2.7299}, {"x": 7.474747475, "y": -2.8986},
          {"x": 7.575757576, "y": -3.0688}, {"x": 7.676767677, "y": -3.2404}, {"x": 7.777777778, "y": -3.4133},
          {"x": 7.878787879, "y": -3.5874}, {"x": 7.97979798, "y": -3.7625}, {"x": 8.080808081, "y": -3.9385},
          {"x": 8.181818182, "y": -4.1152}, {"x": 8.282828283, "y": -4.2924}, {"x": 8.383838384, "y": -4.4700},
          {"x": 8.484848485, "y": -4.6479}, {"x": 8.585858586, "y": -4.8259}, {"x": 8.686868687, "y": -5.0038},
          {"x": 8.787878788, "y": -5.1815}, {"x": 8.888888889, "y": -5.3588}, {"x": 8.98989899, "y": -5.5355},
          {"x": 9.090909091, "y": -5.7116}, {"x": 9.191919192, "y": -5.8869}, {"x": 9.292929293, "y": -6.0612},
          {"x": 9.393939394, "y": -6.2344}, {"x": 9.494949495, "y": -6.4064}, {"x": 9.595959596, "y": -6.5770},
          {"x": 9.696969697, "y": -6.7461}, {"x": 9.797979798, "y": -6.9135}, {"x": 9.898989899, "y": -7.0792},
          {"x": 10.0, "y": -7.2429}
        ],
        "backgroundColor": "rgba(255, 99, 132, 1)",
        "borderColor": "rgba(255, 99, 132, 1)",
        "fill": false,
        "showLine": true,
        "pointRadius": 0
      }
    ]
  },
  "options": {
    "scales": {
      "x": {
        "title": {
          "display": true,
          "text": "x"
        }
      },
      "y": {
        "title": {
          "display": true,
          "text": "y"
        }
      }
    },
    "plugins": {
      "title": {
        "display": true,
        "text": "Locally Weighted Regression (LOWESS)"
      },
      "legend": {
        "display": true
      }
    }
  }
}
```

**Chart Details**:
- **Type**: Scatter for noisy data, line for the LOWESS fit.
- **Data**: The noisy data points are from the synthetic dataset (`sin(x) + noise`). The smoothed points are computed using the custom LOWESS implementation.
- **Styling**: Blue scatter points for noisy data with transparency (`rgba(54, 162, 235, 0.5)`), red line for the LOWESS fit (`rgba(255, 99, 132, 1)`).
- **Axes and Title**: Labeled axes and a title for clarity.

---

### **5. Why This Dataset and Approach?**

**Dataset Suitability**:
- The synthetic dataset (\( y = \sin(x) + \text{noise} \)) is ideal because:
  - It has a clear, non-linear trend (sine wave) that tests LOWESS’s ability to capture complex patterns.
  - The noise simulates real-world data imperfections, making it a realistic test case.
  - It’s simple and controlled, allowing us to focus on the algorithm’s behavior.

**Why LOWESS?**
- LOWESS is robust to outliers and doesn’t require specifying a global model, making it suitable for datasets with unknown or complex trends.
- The `frac=0.2` parameter balances smoothness and sensitivity, capturing the sine wave’s oscillations without overfitting to the noise.

**Comparison with Other Methods**:
- **Linear Regression**: Would fail to capture the sine wave’s non-linearity.
- **Polynomial Regression**: Could fit the sine wave but requires choosing the degree and may overfit.
- LOWESS adapts locally, making it more flexible.

---

### **6. Additional Considerations**

- **Tuning `frac`**: Experimenting with different `frac` values (e.g., 0.1, 0.5) can adjust the smoothness. Smaller values make the curve more wiggly, larger values make it smoother.
- **Computational Cost**: LOWESS is computationally intensive because it fits a regression for each point. For large datasets, optimized implementations (like `statsmodels`) are preferred.
- **Real-World Applications**: LOWESS is used in time-series analysis, signal processing, and data visualization to smooth noisy data while preserving trends.

---

### **7. Summary**

- **LOWESS Basics**: A non-parametric method that fits local regressions to smooth noisy data, using a weighted kernel (e.g., tricube).
- **Provided Code**: Uses `statsmodels` to apply LOWESS to a noisy sine wave dataset, visualizing the result with `matplotlib`.
- **Custom Implementation**: Demonstrates how LOWESS works by implementing it from scratch with a tricube kernel and weighted linear regression.
- **Visualization**: A Chart.js chart shows the noisy data and smoothed curve, confirming that LOWESS effectively captures the sine wave’s trend.
- **Dataset Choice**: The synthetic sine wave with noise is appropriate for testing LOWESS’s ability to handle non-linear, noisy data.

If you’d like to experiment with a different dataset, adjust the `frac` parameter, or explore another aspect of LOWESS, let me know!